# A Deeper Look Into Commodity Market Impacts

## Import Packages

In [1]:
# Web - Scraping and API Requests
import requests


# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json

# Database Connection
from sqlalchemy import create_engine

# SQL Querying
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

## Other Setup

In [2]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

In [3]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

In [4]:
# this allows one to reload the custom package without having to install it again
%autoreload 1




### Import Custom Packages

In [5]:
# Change the wd
sys.path.insert(0,'../src/')

#Import the packages
from macro_utils import sql_queries as sqlq
from macro_utils import functions as macro

## Connect to the Database

### Finding the correct file directory for the database credentials json with the api key and password


In [6]:
current_dir = os.path.dirname(os.path.abspath("NB01_Extract_Data.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "supabase_credentials.json")

# open the  credentials file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

#### Connect to the SupaBase Engine

In [7]:
# connect to the database
supabase_engine = sqlq.get_supabase_engine(
    user="postgres",
    password=credentials['password'],
    host=credentials['host'],
    port=5432,
    database="postgres"
)

### Create an Empty Table and Fill it with the Right Data

In [8]:
## Drop the table if we wish
with supabase_engine.begin() as connection:
    connection.execute(text(sqlq.DROP_TS_TABLE_SQL_QUERY))

In [9]:
# Create a sample DataFrame with 2 columns: 'yq' (year-quarter) and 'level'
test_data = pd.DataFrame({
    'yq': ['2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1'],
    'level': [100, 110, 105, 115, 120]
})

In [10]:
## Execute the CREATE TABLE query to create a blank table if it doesn't already exist
with supabase_engine.begin() as connection:
    connection.execute(text(sqlq.CREATE_TS_TABLE_SQL_QUERY))

In [11]:


## find the ids (in this case dates) that already exist
with supabase_engine.connect() as conn:
    existing_ids = conn.execute(text("SELECT yq FROM ts_data")).fetchall()
## filter out only the ids that will be unique to the existing table
existing_ids = {row[0] for row in existing_ids}
new_rows = test_data[~test_data['yq'].isin(existing_ids)]



In [12]:
# fill in the data into the table
sqlq.make_table(new_rows, "ts_data", supabase_engine)

### Test if Can be Read Back In Properly

In [14]:
with supabase_engine.connect() as connection:
    test = pd.read_sql(text(sqlq.GET_TS_DATA_SQL_QUERY), connection)
test.head()

,yq,level
0,2023Q1,100.0
1,2023Q2,110.0
2,2023Q3,105.0
3,2023Q4,115.0
4,2024Q1,120.0


## Create and use a Function to Dynamically Create a SQL Table

This should automatically detect the columns and types of the data and create a table with the correct column names and types, with only as much data capacity as needed.

In [ ]:
def infer_sql_type(series) -> str:
    """
    Infers the appropriate SQL data type for a pandas Series.

    Args:
        series: pd.Series
            The pandas Series for which to infer the SQL type.

    Returns:
        str: The inferred SQL data type as a string.
    """
    # Check if the series is of integer type
    if pd.api.types.is_integer_dtype(series):
        return "INTEGER"
    # Check if the series is of float type
    elif pd.api.types.is_float_dtype(series):
        return "REAL"
    # Check if the series is of boolean type
    elif pd.api.types.is_bool_dtype(series):
        return "BOOLEAN"
    # Check if the series is of datetime type
    elif pd.api.types.is_datetime64_any_dtype(series):
        return "TIMESTAMP"
    else:
        # For object types, infer VARCHAR length or fallback to TEXT
        max_len = series.dropna().astype(str).map(len).max()
        return f"VARCHAR({max_len})" if max_len else "TEXT"

def df_to_create_table_sql(df: pd.DataFrame, table_name: str) -> str:
    """
    Generates a SQL CREATE TABLE statement based on the columns and types of a pandas DataFrame.

    Args:
        df: pd.DataFrame
            The DataFrame to analyze.
        table_name: str
            The name of the SQL table to create.

    Returns:
        str: The SQL CREATE TABLE statement as a string.
    """
    cols = []
    # Iterate through each column to infer its SQL type and build column definitions
    for col in df.columns:
        sql_type = infer_sql_type(df[col])
        cols.append(f"{col} {sql_type}")
    # Join all column definitions into a single string
    col_definitions = ",\n    ".join(cols)
    # Format the final CREATE TABLE SQL statement
    return f"CREATE TABLE IF NOT EXISTS {table_name} (\n    {col_definitions}\n);"



('CREATE TABLE IF NOT EXISTS test_data (\n'
 '    yq VARCHAR(6),\n'
 '    level INTEGER\n'
 ');')


### Use this Function to Create the Table


In [21]:
CREATE_TABLE_SQL_QUERY = df_to_create_table_sql(test_data, "test_data")
CREATE_TABLE_SQL_QUERY

'CREATE TABLE IF NOT EXISTS test_data (\n    yq VARCHAR(6),\n    level INTEGER\n);'

In [22]:
## Execute the CREATE TABLE query to create a blank table if it doesn't already exist
with supabase_engine.begin() as connection:
    connection.execute(text(CREATE_TABLE_SQL_QUERY))

In [23]:
# fill in the data into the table
sqlq.make_table(new_rows, "test_data", supabase_engine)

## Analyse the Data